# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')
from sklearn import svm
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from matplotlib.pylab import rc, plot

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.sample(10)

,text,toxic
128167,Hi to both! I found the problem and I fixed bo...,0
44691,Groups of Death in 2006 \n\nThis was discussed...,0
79738,I use ST to create songs. I've mostly created ...,0
92047,"Mangum \n\nAs you know, agenda-pushing editors...",0
9888,hello \n\nPlease do not file false reports at ...,0
92188,"Ok, without personal attacks: They never devel...",0
95273,A 34 year old defendant in the V Festival murd...,0
140950,"""\nYes there is. WP:ENG: """"Citations to non-En...",0
32011,"""\n\n Jdorney really needs to grow up here. \n...",0
30322,"""\n\n A barnstar for you! \n\n The Original B...",0


In [5]:
df.isna().sum()

text     0
toxic    0
dtype: int64

In [6]:
df.duplicated().sum()

0

Оценим сбалансированность данных:

In [7]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [8]:
df['toxic'].value_counts() / df['toxic'].count()

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Как мы видим данные несбалансированны. Количество положительных ответов - **~10%**

In [9]:
from pymystem3 import Mystem
m = Mystem()


Создадим функцию `lemmatize`, которая выполняет:
- делает все слова в нижнем регистре,
- лемматизирует текст,
- модулем `re`, оставляет в тексте только латинские символы и пробелы.


In [10]:
def lemmatize(text):
    text = text.lower()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
    text = re.sub(r'[^a-zA-Z]', ' ', lemm_text)
    text = ' '.join(text.split())
    return text
    

In [11]:
%%time
df['lemm_text'] = df['text'].apply(lemmatize)

CPU times: user 1min 4s, sys: 12.6 s, total: 1min 17s
Wall time: 2min 43s


In [12]:
df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [13]:
features = df['lemm_text']
target = df['toxic']

In [14]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345, stratify=target)

In [15]:
print(features.shape)
print(features_train.shape)
print(features_test.shape)

(159571,)
(127656,)
(31915,)


In [16]:
features_train.shape[0] + features_test.shape[0]

159571

Методом TF-IDF оценим важность слов:

In [17]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) # stopwords - список стоп-слов
train_tf_idf = count_tf_idf.fit_transform(features_train.values.astype('U'))

In [19]:
train_tf_idf.shape

(127656, 148931)

In [20]:
test_tf_idf = count_tf_idf.transform(features_test.values.astype('U'))

In [21]:
test_tf_idf.shape

(31915, 148931)

In [22]:
train_tf_idf.shape[0] + test_tf_idf.shape[0]

159571

### Вывод

Даннные состоят из 159571 строк и 2 столбцов.  
Пропусков и дубликатов нет.  
Данные несбалансированны, количество положительных ответов - **~10%**.  
Из данных выделена тестовая выборка (20%) для оценки качества моделей.  
Проведена подготовка данных (лемматизация, замена латинскими буквами и пробелами, TF-IDF)

## Обучение

Проведем обучение следующих моделей:
1. LogisticRegressionCV,
2. SGDClassifier,
3. RidgeClassifierCV, 
4. RandomForestClassifier.  


Требуется найти лучшую модель с метрикой f1 более 0,75.  
Учитывая несбалансиррованность классов, обучение будем проводить со значением параметра `class_weight='balanced'`.

### LogisticRegressionCV

In [ ]:
%%time
lrCV = LogisticRegressionCV(random_state=12345, class_weight='balanced', cv=3, n_jobs=-1)
lrCV.fit(train_tf_idf, target_train)
pred_lrCV = lrCV.predict(test_tf_idf)
f1_pred_lrCV = f1_score(target_test, pred_lrCV)
f1_pred_lrCV

In [ ]:
print(classification_report(target_test, pred_lrCV))

In [ ]:
confusion_matrix(target_test, pred_lrCV)

Модель LogisticRegression, корректно находит негативыне комментарии в 75% случаев, доля найденых негативных комментариев 81%.

### SGDClassifier

In [ ]:
%%time
clf_sgdc = SGDClassifier(max_iter=1000, class_weight='balanced', n_jobs=-1, random_state=12345)
clf_sgdc.fit(train_tf_idf, target_train)
pred_sgdc = clf_sgdc.predict(test_tf_idf)
f1_sgdc = f1_score(target_test, pred_sgdc)
f1_sgdc

In [ ]:
print(classification_report(target_test, pred_sgdc))

In [ ]:
confusion_matrix(target_test, pred_sgdc)

Модель SGDClassifier, корректно находит негативыне комментарии в 67% случаев, доля найденых негативных комментариев 84%.

### RidgeClassifierCV

In [ ]:
%%time
clf_rcfCV = RidgeClassifierCV(class_weight='balanced', cv=3)
clf_rcfCV.fit(train_tf_idf, target_train)
pred_rcf = clf_rcfCV.predict(test_tf_idf)
f1_pred_rcf = f1_score(target_test, pred_rcf)
f1_pred_rcf

In [ ]:
print(classification_report(target_test, pred_rcf))

In [ ]:
confusion_matrix(target_test, pred_rcf)

Модель RidgeClassifier, корректно находит негативыне комментарии в 18% случаев, доля найденых негативных комментариев 82%.

### RandomForestClassifier

In [ ]:
%%time
clf_rfr = RandomForestClassifier(n_estimators=100, random_state=12345, class_weight='balanced', n_jobs=-1)
clf_rfr.fit(train_tf_idf, target_train)
pred_rfr = clf_rfr.predict(test_tf_idf)
f1_rfr = f1_score(target_test, pred_rfr)
f1_rfr


In [ ]:
print(classification_report(target_test, pred_rfr))

In [ ]:
confusion_matrix(target_test, pred_rfr)

Модель RandomForestClassifier, корректно находит негативыне комментарии в 94% случаев, доля найденых негативных комментариев 64%.

In [ ]:
data_training = {'f1_score': [f1_pred_lrCV.round(2), f1_sgdc.round(2), f1_pred_rcf.round(2), f1_rfr.round(2)]}
index = ['LinearRegressionCV', 'SGDClassifier', 'RidgeClassifierCV', 'RandomForestClassifier']
model_analysis = pd.DataFrame(data_training, index=index)
model_analysis = model_analysis.sort_values(by='f1_score', ascending=False)
model_analysis

## Выводы

Даннные состоят из 159571 строк и 2 столбцов.  
Пропусков и дубликатов нет.  
Данные несбалансированны, количество положительных ответов - **~10%**.  
Из данных выделена тестовая выборка (20%) для оценки качества моделей.  
Проведена подготовка данных (лемматизация, замена латинскими буквами и пробелами, TF-IDF)  


| Models        | f1_score| 
| ------------- |------------------| 
| LogisticRegressionCV |0.78|
| SGDClassifier |0.74| 
| RandomForestClassifier |0.64| 
| RidgeClassifierCV |0.29| 


Лучший результат метрики f1 score равное **~0,78** показала модель - **LogisticRegression**.  
Наилучший показатель по **recall** равный **0,84** у модели **SGDClassifier**, также у нее отличные показатели по времени обучения.
